<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/2010.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

# Agora, 'dados' contém os dados do arquivo XLSX


In [17]:
# Exibe as primeiras linhas do DataFrame
print(dados.head())


    ID          Nome Código    Setor EconômicoB3    VM30Abr11  PL Dez 2010  \
0  389    Wetzel S/A  MWET4     Bens industriais   77758.1000       1414.0   
1  152     Excelsior  BAUH4  Consumo não cíclico   12011.1106       1971.0   
2  126  Dtcom Direct  DTCY3     Bens industriais   67358.1720       5871.0   
3  133    Embpar S/A  EPAR3     Bens industriais  176122.6600      17579.0   
4   74       Cambuci  CAMB3      Consumo cíclico  112728.0000      34431.0   

   LLDez 2010  ATDez 2010  PTDez 2010  VM30Abr11esc  PL Dez 2010esc  \
0      -731.0    194929.0    193515.0      0.544898        0.009909   
1     -4642.0     30461.0     28490.0      0.469624        0.077064   
2      -508.0     16764.0     10893.0      5.809744        0.506383   
3     -3281.0    598100.0    580521.0      0.304585        0.030401   
4      9449.0    178747.0    144316.0      0.689046        0.210458   

   LLDez 2010esc  ATDez 2010esc  PTDez 2010esc  ATDez 2009  
0      -0.005123       1.365986       1.356

REGRESSÃO SEM ESCALA

In [22]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL Dez 2010', 'LLDez 2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:              VM30Abr11   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                     2419.
Date:                Thu, 14 Sep 2023   Prob (F-statistic):          2.26e-134
Time:                        11:46:47   Log-Likelihood:                -3250.8
No. Observations:                 190   AIC:                             6508.
Df Residuals:                     187   BIC:                             6517.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.758e+06   4.87e+05      3.608      

tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [18]:
from scipy.stats.mstats import winsorize

nivel_winsorizacao = 0.01  # 1% de winsorização

# Aplica a winsorização às variáveis
dados['PL Dez 2010_winsorizada'] = winsorize(dados['PL Dez 2010esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['LLDez 2010_winsorizada'] = winsorize(dados['LLDez 2010esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['VM30Abr11_winsorizada'] = winsorize(dados['VM30Abr11esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
# Variáveis winsorizadas agora contêm os valores winsorizados

In [19]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL Dez 2010_winsorizada', 'LLDez 2010_winsorizada']]
y = dados['VM30Abr11_winsorizada']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

                              OLS Regression Results                             
Dep. Variable:     VM30Abr11_winsorizada   R-squared:                       0.311
Model:                               OLS   Adj. R-squared:                  0.304
Method:                    Least Squares   F-statistic:                     42.23
Date:                   Thu, 14 Sep 2023   Prob (F-statistic):           7.36e-16
Time:                           11:43:21   Log-Likelihood:                -320.23
No. Observations:                    190   AIC:                             646.5
Df Residuals:                        187   BIC:                             656.2
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

Em relação aos resultados apresentados no seu modelo de regressão:

O valor do teste de Durbin-Watson é 1.676, o que sugere uma leve autocorrelação positiva nos resíduos, mas não muito forte.
O valor-p associado ao teste de Jarque-Bera é praticamente zero (0.00), o que indica que os resíduos não seguem uma distribuição normal.

Interpretação DW:

Valor DW ≈ 2: Ausência de autocorrelação significativa nos resíduos (boa notícia).
Valor DW < 2: Autocorrelação positiva nos resíduos (erros adjacentes são correlacionados positivamente).
Valor DW > 2: Autocorrelação negativa nos resíduos (erros adjacentes são correlacionados negativamente).

teste de heterocedasticidade de White. Se o valor-p for menor que um nível de significância escolhido (por exemplo, 0,05), você pode rejeitar a hipótese nula de homocedasticidade

In [23]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

# Calcule os resíduos do modelo
residuos = modelo.resid

# Realize o teste de White para heterocedasticidade
teste_white = het_white(residuos, X)

# Imprima os resultados do teste
print("Estatística do teste de White:", teste_white[0])
print("Valor-p do teste de White:", teste_white[1])

Estatística do teste de White: 88.5560770784808
Valor-p do teste de White: 2.6679629712877945e-18


erro absoluto médio (MAE) e erro médio quadrático (MSE)

In [24]:
import statsmodels.api as sm
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Faça previsões com o modelo
previsoes = modelo.predict(X)

# Calcule o Median Absolute Error (MedAE)
medae = np.median(np.abs(y - previsoes))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

# Calcule o Erro Absoluto Médio (MAE)
mae = np.mean(np.abs(y - previsoes))

# Calcule o Erro Médio Quadrático (MSE)
mse = np.mean((y - previsoes) ** 2)

# Calcule a Raiz do Erro Quadrático Médio (RMSE)
rmse = np.sqrt(mse)

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)
print("Erro Absoluto Médio (MAE):", mae)
print("Erro Médio Quadrático (MSE):", mse)
print("Raiz do Erro Quadrático Médio (RMSE):", rmse)


Median Absolute Error (MedAE): 1727924.2895822893
MedAE escalado pelo valor (ou preço) em percentagem: 19.958420082487134
Erro Absoluto Médio (MAE): 2900067.5475697857
Erro Médio Quadrático (MSE): 42516419228227.7
Raiz do Erro Quadrático Médio (RMSE): 6520461.580918003


Nesse caso específico, o valor de 17.74% indica que, em média, as previsões do modelo estão erradas em cerca de 17.74% do valor médio (ou preço) da variável de resposta. Isso ajuda a ter uma ideia do tamanho relativo dos erros do modelo em relação ao valor médio dos dados.

minimos quadrados ponderados, com pesos com base na variância dos erros

In [ ]:
import statsmodels.api as sm
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL Dez 2010', 'LLDez 2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Calcule os resíduos (erros)
residuos = modelo.resid

# Calcule a variância dos resíduos
variancia_residuos = np.var(residuos)

# Calcule os pesos com base na variância dos resíduos (inverso da variância)
pesos = 1 / variancia_residuos

# Crie o modelo de regressão linear ponderada (WLS) com os pesos calculados
modelo_wls = sm.WLS(y, X, weights=pesos).fit()

# Visualize os resultados do modelo ponderado
print(modelo_wls.summary())


                            WLS Regression Results                            
Dep. Variable:              VM30Abr11   R-squared:                       0.959
Model:                            WLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     2500.
Date:                Wed, 13 Sep 2023   Prob (F-statistic):          1.27e-148
Time:                        16:42:07   Log-Likelihood:                -3711.2
No. Observations:                 216   AIC:                             7428.
Df Residuals:                     213   BIC:                             7439.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.755e+06   4.94e+05      3.554      

ESTIMAÇÃO FORA DA AMOSTRA

In [ ]:
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL Dez 2010', 'LLDez 2010']]
y = dados['VM30Abr11']

# Converta os dados para matrizes NumPy
X = X.to_numpy()
y = y.to_numpy()

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X)

# Defina a validação cruzada de 10-fold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar as métricas em cada fold
r2_scores = []
mae_scores = []
mse_scores = []
rmse_scores = []  # Adicionado para armazenar RMSE

# Realize a validação cruzada e calcule as métricas em cada fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    modelo_fold = modelo.fit()
    y_pred = modelo_fold.predict(X_test)

    r2_scores.append(r2_score(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))
    mse_scores.append(mean_squared_error(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))  # Cálculo do RMSE

# Calcule as médias das métricas em todos os folds
mean_r2_score = np.mean(r2_scores)
mean_mae = np.mean(mae_scores)
mean_mse = np.mean(mse_scores)
mean_rmse = np.mean(rmse_scores)  # Média do RMSE

# Imprima as médias das métricas
print(f'Média do R²: {mean_r2_score}')
print(f'Média do Erro Absoluto Médio (MAE): {mean_mae}')
print(f'Média do Erro Médio Quadrático (MSE): {mean_mse}')
print(f'Média da Raiz do Erro Médio Quadrático (RMSE): {mean_rmse}')


Média do R²: 0.792669429937137
Média do Erro Absoluto Médio (MAE): 2779057.4455175055
Média do Erro Médio Quadrático (MSE): 51603169213414.05
Média da Raiz do Erro Médio Quadrático (RMSE): 6339657.325339642
